# Fabric Date Table Generator (PySpark Approach)

This notebook generates a dynamic date table for use in Microsoft Fabric Lakehouse and Power BI Direct Lake. It is structured for clarity, flexibility, and maintainability.

## Parameters

All parameters are defined here for easy customization. Adjust these as needed for your scenario.

- **start_date**: First date in the table (YYYY-MM-DD)
- **end_date**: Last date in the table (YYYY-MM-DD)
- **fiscal_year_start_month**: Month (1-12) when the fiscal year starts
- **holiday_dates**: List of holiday dates (YYYY-MM-DD) to flag in the table

In [1]:
# Parameters
from datetime import date

start_date = date(2015, 1, 1)
end_date = date(2030, 12, 31)
fiscal_year_start_month = 7  # July
holiday_dates = [
    date(2025, 1, 1),
    date(2025, 12, 25)
    ]

StatementMeta(, ad2d7c8f-795e-4055-8a1c-b5326240b671, 3, Finished, Available, Finished)

## Generate Date Range

We use PySpark to generate the date range directly for scalability. This approach is efficient for large ranges and avoids pandas bottlenecks.

In [2]:
from pyspark.sql.functions import sequence, to_date, col, explode, expr
from pyspark.sql.types import DateType

# Create a Spark DataFrame with start and end dates
spark_df = spark.createDataFrame([(start_date, end_date)], ["start", "end"])

# Generate date range using sequence with INTERVAL
date_sdf = spark_df.select(
    explode(expr("sequence(start, end, interval 1 day)")).alias("Date")
).withColumn("Date", to_date(col("Date")))

# Show results
print(f"Generated {date_sdf.count()} rows for date range.")
date_sdf.show(5)


StatementMeta(, ad2d7c8f-795e-4055-8a1c-b5326240b671, 4, Finished, Available, Finished)

Generated 5844 rows for date range.
+----------+
|      Date|
+----------+
|2015-01-01|
|2015-01-02|
|2015-01-03|
|2015-01-04|
|2015-01-05|
+----------+
only showing top 5 rows



## Add Date Columns

We add calendar, fiscal, and time intelligence columns. All logic is vectorized for performance.

In [3]:
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, weekofyear, quarter, date_format, when, lit

df = date_sdf \
    .withColumn("Year", year(col("Date"))) \
    .withColumn("Month", month(col("Date"))) \
    .withColumn("Day", dayofmonth(col("Date"))) \
    .withColumn("Quarter", quarter(col("Date"))) \
    .withColumn("MonthName", date_format(col("Date"), "MMMM")) \
    .withColumn("DayOfWeek", dayofweek(col("Date"))) \
    .withColumn("WeekOfYear", weekofyear(col("Date")))

# Fiscal Year/Quarter/Month logic
df = df \
    .withColumn("FiscalYear",
        when(col("Month") >= fiscal_year_start_month, col("Year") + 1).otherwise(col("Year"))
    ) \
    .withColumn("FiscalMonth", ((col("Month") - fiscal_year_start_month + 12) % 12 + 1)) \
    .withColumn("FiscalQuarter", ((col("FiscalMonth") - 1) / 3 + 1).cast("int"))

# Holiday flag
from pyspark.sql.functions import array, lit as spark_lit
df = df.withColumn("IsHoliday", col("Date").cast("string").isin([d.isoformat() for d in holiday_dates]))

# Current day/month/year flags
from datetime import datetime
today = datetime.today().date()
df = df \
    .withColumn("IsCurrentDay", col("Date") == spark_lit(today.isoformat())) \
    .withColumn("IsCurrentMonth", (col("Year") == today.year) & (col("Month") == today.month)) \
    .withColumn("IsCurrentYear", col("Year") == today.year)

df.show(5)

StatementMeta(, ad2d7c8f-795e-4055-8a1c-b5326240b671, 5, Finished, Available, Finished)

+----------+----+-----+---+-------+---------+---------+----------+----------+-----------+-------------+---------+------------+--------------+-------------+
|      Date|Year|Month|Day|Quarter|MonthName|DayOfWeek|WeekOfYear|FiscalYear|FiscalMonth|FiscalQuarter|IsHoliday|IsCurrentDay|IsCurrentMonth|IsCurrentYear|
+----------+----+-----+---+-------+---------+---------+----------+----------+-----------+-------------+---------+------------+--------------+-------------+
|2015-01-01|2015|    1|  1|      1|  January|        5|         1|      2015|          7|            3|    false|       false|         false|        false|
|2015-01-02|2015|    1|  2|      1|  January|        6|         1|      2015|          7|            3|    false|       false|         false|        false|
|2015-01-03|2015|    1|  3|      1|  January|        7|         1|      2015|          7|            3|    false|       false|         false|        false|
|2015-01-04|2015|    1|  4|      1|  January|        1|         

## Write to Lakehouse Table

We save the date table to the Lakehouse in Delta format. This enables Direct Lake access in Power BI for real-time analytics.

- Table name: `date_table`
- Mode: `overwrite` (regenerates the table each run)

In [4]:
df.write.format("delta").mode("overwrite").saveAsTable("date_table")
print("Date table written to Lakehouse as 'date_table'.")

StatementMeta(, ad2d7c8f-795e-4055-8a1c-b5326240b671, 6, Finished, Available, Finished)

Date table written to Lakehouse as 'date_table'.


## Testing & Validation

We include basic tests to validate fiscal year logic and print sample output for review.

In [5]:
# Test: Fiscal year boundary
import pandas as pd
test_date = pd.Timestamp(year=2025, month=fiscal_year_start_month, day=1)
fy = test_date.year + (1 if test_date.month >= fiscal_year_start_month else 0)
row = df.filter(df.Date == test_date.isoformat()).select("FiscalYear").collect()
if row:
    assert row[0][0] == fy, f"FiscalYear logic failed for {test_date}"
print("FiscalYear logic test passed.")

# Show first and last few rows
df.orderBy("Date").show(3)
df.orderBy(col("Date").desc()).show(3)

StatementMeta(, ad2d7c8f-795e-4055-8a1c-b5326240b671, 7, Finished, Available, Finished)

FiscalYear logic test passed.
+----------+----+-----+---+-------+---------+---------+----------+----------+-----------+-------------+---------+------------+--------------+-------------+
|      Date|Year|Month|Day|Quarter|MonthName|DayOfWeek|WeekOfYear|FiscalYear|FiscalMonth|FiscalQuarter|IsHoliday|IsCurrentDay|IsCurrentMonth|IsCurrentYear|
+----------+----+-----+---+-------+---------+---------+----------+----------+-----------+-------------+---------+------------+--------------+-------------+
|2015-01-01|2015|    1|  1|      1|  January|        5|         1|      2015|          7|            3|    false|       false|         false|        false|
|2015-01-02|2015|    1|  2|      1|  January|        6|         1|      2015|          7|            3|    false|       false|         false|        false|
|2015-01-03|2015|    1|  3|      1|  January|        7|         1|      2015|          7|            3|    false|       false|         false|        false|
+----------+----+-----+---+-------

## Notes & Best Practices

- All calculations are performed upstream for Direct Lake compatibility.
- The notebook is parameterized for easy reuse.
- For very large date ranges, Spark generation is scalable.
- The output table can be shared across Power BI datasets for consistency.
- For maintenance, update the end date and holiday list as needed.